In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import os

In [2]:
api_key = "1c269e5572d94e64b8d165935251302"  # Reemplaza con tu clave de API
ciudad = "Marbella"
fecha_fin = datetime.now()  # Fecha actual
fecha_inicio = fecha_fin - timedelta(days=365)  # Último año
delta = timedelta(days=1)

datos_historicos = []

In [3]:
while fecha_inicio <= fecha_fin:
    url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={ciudad}&dt={fecha_inicio.strftime('%Y-%m-%d')}"
    respuesta = requests.get(url)
    
    # Verificar si la solicitud fue exitosa
    if respuesta.status_code == 200:
        datos = respuesta.json()
        
        # Verificar si la clave 'forecast' está en la respuesta
        if 'forecast' in datos and 'forecastday' in datos['forecast'] and len(datos['forecast']['forecastday']) > 0:
            for hora in datos['forecast']['forecastday'][0]['hour']:
                datos_historicos.append({
                    'Fecha': hora['time'],
                    'Temperatura_C': hora['temp_c'],
                    'Sensacion_termica_C': hora['feelslike_c'],
                    'Humedad': hora['humidity'],
                    'Precipitaciones_mm': hora['precip_mm'],
                    'Presion_mb': hora['pressure_mb'],
                    'Viento_kph': hora['wind_kph'],
                    'is_day': hora['is_day'],
                    'Descripcion': hora['condition']['text'],
                    'Probabilidad_lluvia': hora['chance_of_rain'],
                    'Llueve': hora['will_it_rain'],
                })
        else:
            print(f"No se encontraron datos de pronóstico para {fecha_inicio.strftime('%Y-%m-%d')}")
    else:
        print(f"Error en la solicitud para {fecha_inicio.strftime('%Y-%m-%d')}: {respuesta.status_code}")
        print(f"Respuesta de la API: {respuesta.text}")  # Imprimir la respuesta de la API para más detalles
    
    fecha_inicio += delta

In [4]:
# Crear DataFrame solo si hay datos
if datos_historicos:
    df_historico = pd.DataFrame(datos_historicos)
else:
    print("No se encontraron datos históricos.")

In [5]:
df_historico.shape

(8784, 11)

In [6]:
df_historico.head(5)

,Fecha,Temperatura_C,Sensacion_termica_C,Humedad,Precipitaciones_mm,Presion_mb,Viento_kph,is_day,Descripcion,Probabilidad_lluvia,Llueve
0,2024-02-25 00:00,10.8,9.7,71,0.0,1023.0,9.4,0,Clear,0,0
1,2024-02-25 01:00,10.3,9.1,71,0.0,1023.0,9.4,0,Clear,0,0
2,2024-02-25 02:00,9.5,8.2,71,0.0,1023.0,8.6,0,Clear,0,0
3,2024-02-25 03:00,9.4,8.1,72,0.0,1022.0,9.0,0,Clear,0,0
4,2024-02-25 04:00,9.3,8.2,72,0.0,1023.0,8.3,0,Clear,0,0


In [7]:
df_historico.tail(5)

,Fecha,Temperatura_C,Sensacion_termica_C,Humedad,Precipitaciones_mm,Presion_mb,Viento_kph,is_day,Descripcion,Probabilidad_lluvia,Llueve
8779,2025-02-24 19:00,12.0,12.3,69,0.0,1028.0,4.3,1,Sunny,0,0
8780,2025-02-24 20:00,11.7,11.3,70,0.0,1028.0,6.5,0,Overcast,0,0
8781,2025-02-24 21:00,11.9,11.1,70,0.0,1029.0,9.0,0,Clear,0,0
8782,2025-02-24 22:00,12.2,11.4,70,0.0,1029.0,9.4,0,Clear,0,0
8783,2025-02-24 23:00,12.2,11.8,69,0.0,1029.0,6.8,0,Clear,0,0


In [ ]:
# Obtener la ruta absoluta del directorio actual (donde se encuentra el notebook)
current_dir = os.getcwd()
# Navegar a la carpeta padre
parent_dir = os.path.dirname(current_dir)
parent_dir = os.path.dirname(parent_dir)
# Construir la ruta a la carpeta data/raw
data_raw_path = os.path.join(parent_dir, 'data', 'raw','weatherApi')

os.chdir(data_raw_path)

In [9]:
# Verificar si el archivo existe
file_path = os.path.join(data_raw_path, 'df_Marbella_hourly.csv')

In [10]:
if os.path.exists(file_path):
    # Leer el archivo en un DataFrame
    df_nuevo = pd.read_csv(file_path)
    # Concatenar con el DataFrame histórico (asumiendo que df_historico ya está definido)
    df_historico = pd.concat([df_historico, df_nuevo], ignore_index=True)
    # Eliminar duplicados
    df_historico = df_historico.drop_duplicates()
    
    # Guardar el DataFrame resultante en el mismo archivo
    df_historico.to_csv(file_path, index=False)
    print("El archivo ha sido actualizado correctamente.")
else:
    print("El archivo df_Marbella_hourly.csv no existe en la ruta especificada.")
    df_historico.to_csv(file_path, index=False)

El archivo ha sido actualizado correctamente.
